In [7]:
import folium
import sys
sys.path.append('..')
import flood_tool as ft
import pandas as pd
from IPython.display import display
import geopandas

In [ ]:
pt=pd.read_csv('resources/postcodes_labelled.csv')
pt=pt.dropna()
typical_path = "example_data/typical_day.csv"
wet_path = "example_data/wet_day.csv"
station_path = "resources/stations.csv"

In [ ]:
ft.plot_risk_map_latlon_fig(pt,dx=0.01,figsize=[20,5])

In [ ]:
ft.plot_historical_flood_map_latlon_fig(pt,dx=0.01,figsize=[20,5])

In [ ]:
sector_path='../flood_tool/resources/sector_data.csv'
postcode_path='../flood_tool/resources/postcodes_labelled.csv'
postcode_proverty=ft.process_proverty(sector_path,postcode_path)
ft.plot_postcode_headcount_map_latlon_fig(postcode_proverty,dx=0.01,figsize=[20,5])

In [ ]:
ft.plot_postcode_household_map_latlon_fig(postcode_proverty,dx=0.01,figsize=[20,5])

In [ ]:
ft.plot_houseprice_map_latlon_fig(pt,dx=0.01,figsize=[20,5])

In [ ]:
ft.potential_map(typical_path,wet_path,station_path)

In [ ]:
#economic risk map


In [ ]:
#human risk map


In [ ]:
#live data


In [ ]:
ft.plot_map(postcode_file='resources/postcodes_labelled.csv',
        station_file= 'resources/stations.csv',
        typical_file='example_data/typical_day.csv',
        wet_file='example_data/wet_day.csv')

In [ ]:
labelled_postcodes = pd.read_csv('../flood_tool/resources/postcodes_labelled.csv')
labelled_postcodes['latitude'], labelled_postcodes['longitude'] = ft.get_gps_lat_long_from_easting_northing(
    labelled_postcodes['easting'], labelled_postcodes['northing'], rads=False, dms=False
    )
labelled_postcodes

In [ ]:
import folium
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors



def update_map(R_min_filter, R_max_filter):
    m = folium.Map(location=[labelled_postcodes["latitude"].mean(), labelled_postcodes["longitude"].mean()], zoom_start=6)
    norm = mcolors.Normalize(vmin=labelled_postcodes['riskLabel'].min(), vmax=labelled_postcodes['riskLabel'].max())
    cmap = plt.cm.coolwarm
    for idx, row in labelled_postcodes.iterrows():
        if R_min_filter <= row['riskLabel'] <= R_max_filter:
            color = mcolors.to_hex(cmap(norm(row['riskLabel']))) 
            popup_text = f"Postcode: {row['postcode']}, Risk-value: {row['riskLabel']},localAuthority: {row['localAuthority']},medianPrice: {row['medianPrice']}"
            folium.Circle(
                location=(row["latitude"], row["longitude"]),
                radius=5,
                fill_opacity=0.3, 
                color=color,
                fill=True,
                fill_color=color,
                popup=folium.Popup(popup_text, parse_html=True)
            ).add_to(m)
    
    display(m)

R_min_widget = widgets.FloatSlider(value=labelled_postcodes['riskLabel'].min(), min=labelled_postcodes['riskLabel'].min(), max=labelled_postcodes['riskLabel'].max(), description='R min:', continuous_update=False)
R_max_widget = widgets.FloatSlider(value=labelled_postcodes['riskLabel'].max(), min=labelled_postcodes['riskLabel'].min(), max=labelled_postcodes['riskLabel'].max(), description='R max:', continuous_update=False)

ui = widgets.VBox([R_min_widget, R_max_widget])
out = widgets.interactive_output(update_map, {'R_min_filter': R_min_widget, 'R_max_filter': R_max_widget})

display(ui, out)